## Блендинг, Стекинг, feature engineering.

### Интуиция: ансамбли

* В рассмотренных нами методах, таких как Bagging, Random Forest, решения принимаются при помощи усреднения отдельных предсказаний.
* Наверное, ошибки отдельных моделей при их композиции должны компенсироваться в сумме.
* Что будет, если отдельные модели "хорошо работают" в отдельных областях, а в некоторых - "плохо"?

<img src="img/0_sep_models.jpg">

Первая мысль, которая нам приходит в голову - усреднить предсказания моделей (взвешенно или нет, не важно). Тогда для примера выше получим следующую картину:

<img src="img/0_ensembling.jpg">

**Альтернатива** - в качестве итогового предсказания в зависимости от "области" брать ответ от той модели, которая там "работает лучше".

Проделаем это руками для примера выше, тогда:

<img src="img/0_ens_1.jpg">

Можно ли это делать автоматически?

### "Умное" ансамблирование

* Решение о том, насколько конкретной модели можно "доверять" принимается другой моделью
* Назовем эту модель *мета-алгоритмом*
* Рассмотрим два наиболее популярных подхода ансамблирования - **stacking (стэкинг)** и **blending (блэндинг)**

## Блендинг

<img src="img/1_0.jpg" height=50% width=60%>

### Алгоритм:
* Даны обучающая выборка $(X, y)$ и тестовая $(X_{test}, y_{test})$
* Хотим использовать $M$ моделей: $b_1(x), b_2(x), \dots, b_M(x)$
* Поделим обучающую выборку на 2 части: $(X_{train}, y_{train})$ и $(X_{meta}, y_{meta})$
* Назовем $(X_{train}, y_{train}) = A$, $(X_{meta}, y_{meta}) = B$, $(X_{test}, y_{test}) = C$
* Для каждой модели $b_i$:
  * Обучим модель $b_i$ на подвыборке A
  * Для каждого объекта из B сделаем предсказание с помощью $b_i$, получим "столбец мета-признаков" матрицы B
  * Для каждого объекта из С сделаем предсказание с помощью $b_i$, получим "столбец мета-признаков" матрицы С
* Получим новую матрицу мета-признаков $B_{meta}$ размером $(N_{obj_B}, M)$, составленную из предсказаний моделей $b_i$ для объектов из B, и матрицу мета-признаков $C_{meta}$ размером $(N_{obj_C}, M)$, составленную из предсказаний моделей $b_i$ для объектов из С
* Обучим мета-алгоритм на подвыборке $B_{meta}$
* Для каждого объекта из $C_{meta}$ сделаем предсказания при помощи мета-алгоритма, это и будут наши ответы.

### Визуализация:

<img src="img/2_b_0.jpg">
<img src="img/2_b_1.jpg">
<img src="img/2_b_2.jpg">
<img src="img/2_b_3.jpg">
<img src="img/2_b_4.jpg">
<img src="img/2_b_5.jpg">
<img src="img/2_b_6.jpg">
<img src="img/2_b_7.jpg">
<img src="img/2_b_8.jpg">
<img src="img/2_b_9.jpg">
<img src="img/2_b_10.jpg">
<img src="img/2_b_11.jpg">
<img src="img/2_b_12.jpg">
<img src="img/2_b_13.jpg">

**Можно и нужно** использовать разные базовые алгоритмы:
* Логистическая регрессия
* KNN
* Random Forest
* etc

В качестве мета-модели не обязательно брать что-то сложное, иногда достаточно обычной линейной модели. Тогда она в буквальном смысле "взвесит" предсказания базовых моделей.

### Недостатки блендинга:

* Для обучения базовых моделей используется не вся выборка
* Базовые модели учатся только на части А
* Мета-модель учится только на части B

## Стекинг (Stacking)

Идея стекинга является обобщением идеи блендинга на случай, когда число разбиений обучающей выборки для построения мета-алгоритма больше 2

* Даны обучающая $(X, y)$ и тестовая выборки $(X_{test}, y_{test})$
* Хотим использовать $M$ моделей: $b_1(x), b_2(x), \dots, b_m(x)$
* Поделим (X, y) на N равных частей (фолдов, как в кросс-валидации)
* Назовем $(X_{train,i}, y_{train,i}) = A_i$ $(i = 1, 2, 3, \dots, N)$, $(X_{test}, y_{test}) = C$
* Для каждого фолда $A_i$:
  * Обучим $M$ моделей на остальных $N-1$ фолдах (то есть на $A_1$, $A_2$, ..., $A_{i-1}$, $A_{i+1}$, ..., $A_N$)
  * Для каждого объекта из $A_i$ сделаем предсказание
  * Получим новую матрицу "мета-признаков" размера $(N_{obj}, M)$
* Каждую из базовых моделей обучаем **на всей выборке А** и делаем предсказания на С, получаем матрицу мета-признаков $C_{meta}$, составленную из предсказаний моделей $b_i$ на тестовой выборке С.
* Обучим мета-алгоритм $b_{meta}$ на выборке $A_{meta}$
* Для каждого объекта из $C_{meta}$ сделаем предсказания при помощи мета-модели $b_{meta}$ - это и будут наши ответы.

### Визуализация:

<img src="img/2_s_0.jpg">
<img src="img/2_s_1.jpg">
<img src="img/2_s_2.jpg">
<img src="img/2_s_3.jpg">
<img src="img/2_s_4.jpg">
<img src="img/2_s_5.jpg">
<img src="img/2_s_6.jpg">
<img src="img/2_s_7.jpg">
<img src="img/2_s_8.jpg">
<img src="img/2_s_9.jpg">

<img src="img/2_s_11.jpg">

http://blog.kaggle.com/2016/12/27/a-kagglers-guide-to-model-stacking-in-practice/

* Можно делать несколько этажей стекинга - тогда получается "мета-алгоритм", "мета-мета-алгоритм" и т.д.
* Нужно подбирать мета-алгоритм и число фолдов - делаем это на валидации
* Не гарантирует 100% повышения качества относительно лучшей модели
* Иногда вместе с мета-признаками используются исходные признаки
* Для обучения базовых моделей может потребоваться разная обработка признаков (нормализация для линейных моделей и т.д.)

### Главный недостаток стекинга

Мета-признаки обучающей и тестовой выборок - **разные**!

* Мета-признаки обучающей выборки получены с помощью моделей, обученных на (N - 1) / N частях выборки.
* Мета-признаки тестовой выборки получены с помощью моделей, обученных на всей train выборке.

### Стекинг и Kaggle

* В спортивном анализе данных стекинг - один из главных инструментов
* При объединении участников в команды делают стекинг решений отдельных участников
* Количество моделей ограничено только временем работы и терпеливостью участника

### Итоги: блендинг и стекинг

**Преимущества**:
* Позволяет очень "дешево" повысить качество
* Хорошо аппроксимирует данные благодаря взвешиванию предсказаний базовых алгоритмов
* Можно распараллелить (на фолды или модели)

**Недостатки**:
* Требуют большого количества данных
* В зависимости от времени работы базовых моделей, долго учатся и долго работают

## Извлечение и преобразование признаков

Извлечение признаков - представление реального или цифрового объекта в виде вектора чисел

Данные бывают:

1. Числовые
2. Дата и время
3. Геоданные
4. Временные ряды
5. Текстовые данные
6. Графические изображения
7. Звук
8. Видео

В качестве демонстрации будем рассматривать датасет из последнего домашнего задания

In [45]:
import pandas as pd

train = pd.read_csv("data/Train.csv")
test = pd.read_csv("data/Test.csv")

train.head()

,id,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,...,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13,price
0,0,2011-01-01,511,1.0,5,67,2,1,NaN,1.0,...,0,0,0,0,0,0,0,0,0,18473000
1,1,2011-01-01,299,0.0,3,45,2,0,20.0,0.0,...,0,0,0,0,0,0,0,0,0,4157000
2,2,2011-01-01,217,NaN,1,61,3,1,10.0,1.0,...,0,0,0,0,0,0,0,0,0,4530000
3,3,2011-01-01,212,NaN,2,51,2,1,15.0,0.0,...,0,0,0,0,0,0,0,0,0,7981000
4,4,2011-01-01,160,1.0,3,60,3,1,25.0,0.0,...,0,0,0,0,0,0,0,0,0,11888000


### Дата и время

Что можно извлечь:
1. Абсолютное время
2. Периодичность (час, время, месяц)
3. Временной интервал до особого события.

Если посмотреть на колонку date, то увидим, что она имеет тип object. Чтобы работать дальше с ней, нам необходимо ее привести к datetime.

In [46]:
train.date = pd.to_datetime(train.date)
test.date = pd.to_datetime(test.date)

Абсолютно время мы уже получили, теперь попробуем извлечь остальные признаки

In [47]:
def date_features(df):
    df["quarter"] = df.date.apply(lambda s: s.quarter)
    df["month"] = df.date.apply(lambda s: s.month)
    df["week"] = df.date.apply(lambda s: s.week)
    df["day"] = df.date.apply(lambda s: s.day)
    df["dayofweek"] = df.date.apply(lambda s: s.dayofweek)
    
    return df

train = date_features(train)
test = date_features(test)

train.head()

,id,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,...,kw10,kw11,kw12,kw13,price,quarter,month,week,day,dayofweek
0,0,2011-01-01,511,1.0,5,67,2,1,NaN,1.0,...,0,0,0,0,18473000,1,1,52,1,5
1,1,2011-01-01,299,0.0,3,45,2,0,20.0,0.0,...,0,0,0,0,4157000,1,1,52,1,5
2,2,2011-01-01,217,NaN,1,61,3,1,10.0,1.0,...,0,0,0,0,4530000,1,1,52,1,5
3,3,2011-01-01,212,NaN,2,51,2,1,15.0,0.0,...,0,0,0,0,7981000,1,1,52,1,5
4,4,2011-01-01,160,1.0,3,60,3,1,25.0,0.0,...,0,0,0,0,11888000,1,1,52,1,5


### Временные ряды

Что можно извлечь:

1. Среднее значение за период
2. Стандартное отклонение за период
3. Тренд за период
4. Количество пиков за период

Для п.1 и п.2 - см. https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html

Извлечение тренда - https://www.statsmodels.org/dev/generated/statsmodels.tsa.seasonal.seasonal_decompose.html

### Признаки бывают:

1. Количественные
2. Порядковые
3. Категориальные
4. Бинарные

### Зачем преобразовывать признаки?

1. Чтобы конкретный алгоритм правильно их интерпретировал
2. Чтобы конкретный алгоритм правильно находил взаимосвязи
3. Чтобы внести априорные значения о наборе данных или свойствах признаков

### Стандартизация

Для каждого признака в наборе вычитаем среднее и делим на его стандартное отклонение

Применяется к количественным, порядковым и бинарным признакам.

Актуально для:

* Линейные модели
* Метод ближайших соседей

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

### Масштабирование

Значения в наборе приводят к диапазону [0, 1]

Аналогично актуально для линейных моделей и KNN

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler

### Монотонные преобразования

Применение монотонного преобразования (логарифмирование, возведение в степень).

Применяется к количественным и качественным признакам.

Актуально для:

* Линейные модели
* Метод ближайших соседей

### Полиномиальные признаки

Заменяем исходный набор признаков полиномом от исходных признаков:

$$(x_1, x_2) \rightarrow (x_1, x_2, x_1^2, x_2^2, x_1x_2)$$

Применяется к количественным, порядковым и бинарным признакам

Актуально для:

* Линейные модели
* Метод ближайших соседей
* Решающие деревья

### Бинаризация
Область значений количественного признака делим на N участков и приводим к виду N признаков.

Применяется к количественным признакам.

Актуально для:
* Линейные модели
* Решающие деревья

### One hot encoding

Представление признака с N категориями как N бинарных признаков.

Актуально для:
* Решающие деревья
* Линейные модели
* KNN

### Хэширование признаков

Не всегда можно рассчитывать, что категориальные признаки не будут принимать новые значения

$$ x_{feature} = hash(x)$$

произвольное значение $\rightarrow$ - натуральное число $0,\dots, N$

## Работа с пропущенными данными

Простые решения:

* Некоторые алгоритмы поддерживают работу с пропусками "из коробки"
* Закодировать пропуски особым значением (0, -999) и т.д.
* Закодировать пропуски типичным значением - среднее, медиана, наиболее частое значение (считать его можно как по всему датасету, так и по каким-то отдельным группам).

Более сложные решения:
* Для временных рядов можно использовать соседние значения (методы bfill, ffill в pandas)
* Использовать модель для заполнения пропусков (KNN, Random Forest)

## Отбор признаков

Зачем?

* Меньше признаков - выше производительность
* Меньше признаков - проще их сбор и преобразование
* Снижение кол-ва признаков может привести как к улучшению, так и к ухудшению качества модели.

### Статистический подход

* Выбрасываем признаки, значение которых константно на train датасете
* Выбрасываем признаки, которые слабо статистически связаны с целевой переменной (например, корреляция).

### Отбор признаков по важности для модели

Смотрим на то, на какие признаки опирается модель при принятии решений.

* Веса признаков в линейной модели
* Как часто происходят сплиты в Random Forest
* Нулевые коэффициенты в l1 регуляризованной линейной модели

### Последовательный отбор признаков

Пусть N - количество признаков.

1. Из полного набора признаков по очереди выбрасываем каждый и таким образом получаем N новых наборов признаков
2. Оцениваем качество модели на каждом из N наборов признаков
3. Выбираем набор с наилучшей точностью
4. Переходим к шагу 1 (опциально, если точность улучшилась).

### Последовательный набор признаков

Пусть дано M основных и N дополнительных признаков.

1. Из N дополнительных признаков по очереди добавляем каждый к основному набору из M признаков, таким образом получаем N наборов признаков.
2. Оцениваем качество модели на каждом из N наборов признаков.
3. Выбираем набор с наилучшей точностью.
4. Переходим на шаг 1 (опционально).